In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#get and parse website
baseSite = "https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/"
response = requests.get(baseSite)
html = response.content
soup = BeautifulSoup(html, 'lxml')
with open('rottenTomatoesLXMLParser', 'wb') as file:
    file.write(soup.prettify('utf-8'))

#getting notable sections of site
divs = soup.find_all('div', {'class': 'col-sm-18 col-full-xs countdown-item-content'})
headings = [div.find("h2") for div in divs]

#scrape elements from site
movieNames = [heading.find('a').string for heading in headings]

years = [heading.find('span', class_ = 'start-year').string for heading in headings]
years = [year.strip('()') for year in years]
years = [int(year) for year in years]

scores = [heading.find('span', class_ = 'tMeterScore').string for heading in headings]

consensus = [div.find('div', {'class': 'info critics-consensus'}) for div in divs]
consensusCommonPhrase = '\nCritics Consensus: '
consensusComPhraseLen = len(consensusCommonPhrase)
consensusText = [con.text[consensusComPhraseLen:].strip() if con.text.startswith(consensusCommonPhrase) else con.text for con in consensus]

directors = [div.find('div', class_ = 'director') for div in divs]
directorsFinal = [None if director.find('a') is None else director.find('a').string for director in directors]

castInfo = [div.find('div', class_ = 'cast') for div in divs]
cast =  []
for c in castInfo:
    castLinks = c.find_all('a')
    castNames = [link.string for link in castLinks]
    result = ', '.join(castNames)
    cast.append(result)

synopsis = [div.find('div', {'class': 'info synopsis'}) for div in divs]
synopStartPhrase = '\nSynopsis: '
synopEndPhrase = '[More]\n'
lenSynopStart = len(synopStartPhrase)
lenSynopEnd = len(synopEndPhrase)
synopsisText = [syn.text[lenSynopStart:-lenSynopEnd].strip() + ('...') if syn.text.startswith(synopStartPhrase) else syn.text for syn in synopsis]

#create dataframe with scraped elements
moviesInfo = pd.DataFrame()
moviesInfo['Movie Title'] = movieNames
moviesInfo['Year'] = years
moviesInfo['Scores'] = scores
moviesInfo['Director'] = directorsFinal
moviesInfo['Synopsis'] = synopsisText
moviesInfo['Cast'] = cast
moviesInfo['Consensus'] = consensusText

#create csv file from dataframe
moviesInfo.to_csv('rottenTomMoviesInfo.csv', index = False, header = True)

#displaying dataframe
pd.set_option('display.max_colwidth', None)
moviesInfo

,Movie Title,Year,Scores,Director,Synopsis,Cast,Consensus
0,Running Scared,1986,63%,Peter Hyams,"Ray and Danny (Gregory Hines, Billy Crystal) are two Chicago police detectives hot on the trail of drug kingpin Julio...","Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer","Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining."
1,Equilibrium,2002,40%,Kurt Wimmer,"In a futuristic world, a regime has eliminated war by suppressing emotions: books, art and music are strictly forbidden and...","Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen",Equilibrium is a reheated mishmash of other sci-fi movies.
2,Hero,2002,94%,Yimou Zhang,"In this visually arresting martial arts epic set in ancient China, an unnamed fighter (Jet Li) is being honored for...","Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Donnie Yen","With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for."
3,Road House,1989,44%,Rowdy Herrington,"The Double Deuce is the meanest, loudest and rowdiest bar south of the Mason-Dixon Line, and Dalton (Patrick Swayze) has...","Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara",Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.
4,Unstoppable,2010,87%,Tony Scott,"When a massive, unmanned locomotive roars out of control, the threat is more ominous than just a derailment. The train...","Denzel Washington, Chris Pine, Rosario Dawson, Kevin Dunn","As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."
...,...,...,...,...,...,...,...
135,Hard-Boiled,1992,92%,John Woo,A cop who loses his partner in a shoot-out with gun smugglers goes on a mission to catch them. In...,"Chow Yun-Fat, Bowie Lam, Philip Chan, Tony Leung Chiu Wai","Boasting impactful action as well as surprising emotional resonance, Hard Boiled is a powerful thriller that hits hard in more ways than one."
136,The Matrix,1999,83%,Lilly Wachowski,"Neo believes that Morpheus, an elusive figure considered to be the most dangerous man alive, can answer his question --...","Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving","Thanks to the Wachowskis' imaginative vision, The Matrix is a smartly crafted combination of spectacular action and groundbreaking special effects."
137,Terminator 2: Judgment Day,1991,91%,James Cameron,"In this sequel set eleven years after ""The Terminator,"" young John Connor (Edward Furlong), the key to civilization's victory over...","Arnold Schwarzenegger, Linda Hamilton, Edward Furlong, Robert Patrick","T2 features thrilling action sequences and eye-popping visual effects, but what takes this sci-fi/ action landmark to the next level is the depth of the human (and cyborg) characters."
138,Die Hard,1988,94%,John McTiernan,New York City policeman John McClane (Bruce Willis) is visiting his estranged wife (Bonnie Bedelia) on Christmas Eve. He joins...,"Bruce Willis, Alan Rickman, Bonnie Bedelia, Reginald VelJohnson",Its many imitators (and sequels) have never come close to matching the taut thrills of the definitive holiday action classic.
